### Monte Carlo Performance Benchmark: Django vs Gin

Kevin Geidel <br>
MSDS 436: Analytics Systems Engineering <br>
Northwestern University <br>
October 27, 2024 <br>

Scripts/utils used to initialize the benchmark experiment

In [1]:
import json
import pandas as pd
from scipy.io.arff import loadarff

In [2]:
# Load the dataset into a pandas dataframe

data = loadarff('../data/dataset_50_tic-tac-toe.arff')
df = pd.DataFrame(data[0])

In [3]:
# Preview the dataset
df

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,Class
0,b'x',b'x',b'x',b'x',b'o',b'o',b'x',b'o',b'o',b'positive'
1,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'x',b'o',b'positive'
2,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'o',b'x',b'positive'
3,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'b',b'b',b'positive'
4,b'x',b'x',b'x',b'x',b'o',b'o',b'b',b'o',b'b',b'positive'
...,...,...,...,...,...,...,...,...,...,...
953,b'o',b'x',b'x',b'x',b'o',b'o',b'o',b'x',b'x',b'negative'
954,b'o',b'x',b'o',b'x',b'x',b'o',b'x',b'o',b'x',b'negative'
955,b'o',b'x',b'o',b'x',b'o',b'x',b'x',b'o',b'x',b'negative'
956,b'o',b'x',b'o',b'o',b'x',b'x',b'x',b'o',b'x',b'negative'


In [4]:
# Use the django app to load the dataset into the SQLite database
import django_init
from tqdm import tqdm
from core.models import EndState

def clean_val(val):
    if val == b'x':
        return 'X'
    elif val == b'o':
        return 'O'
    else:
        return None

for record in tqdm(df.itertuples()):
    EndState.objects.get_or_create(
        top_left = clean_val(record[1]),
        top_middle = clean_val(record[2]),
        top_right = clean_val(record[3]),
        middle_left = clean_val(record[4]),
        middle_middle = clean_val(record[5]),
        middle_right = clean_val(record[6]),
        bottom_left = clean_val(record[7]),
        bottom_middle = clean_val(record[8]),
        bottom_right = clean_val(record[9]),
        defaults={'x_wins_flag': True if record[10]==b'positive' else False}
    )

958it [00:01, 921.18it/s]


In [5]:
# We can view the queryset of records from the DB in a DF to confirm transfer
# (NOTE: The pk of the EndState table becomes an 11th attribute)
pd.DataFrame(EndState.objects.values())

,id,top_left,top_middle,top_right,middle_left,middle_middle,middle_right,bottom_left,bottom_middle,bottom_right,x_wins_flag
0,1917,X,X,X,X,O,O,X,O,O,True
1,1918,X,X,X,X,O,O,O,X,O,True
2,1919,X,X,X,X,O,O,O,O,X,True
3,1920,X,X,X,X,O,O,O,None,None,True
4,1921,X,X,X,X,O,O,None,O,None,True
...,...,...,...,...,...,...,...,...,...,...,...
953,2870,O,X,X,X,O,O,O,X,X,False
954,2871,O,X,O,X,X,O,X,O,X,False
955,2872,O,X,O,X,O,X,X,O,X,False
956,2873,O,X,O,O,X,X,X,O,X,False
